In [7]:
import cv2
import numpy as np
import os
import pandas as pd
from tqdm import tqdm



def localize_pacman(image_path, pacman_colors=[(210, 164, 74), (142, 142, 142)], background_color=(228, 111, 111), color_tolerance=5, exclude_bottom_pixels=40, min_contour_area=16, max_contour_area=50, max_aspect_ratio=2.0):
    global output_df  # Use the global dataframe

    # Load the image
    image = cv2.imread(image_path)

    # Exclude the bottom pixels from the image
    height, width, _ = image.shape
    image = image[:-exclude_bottom_pixels, :]

    # Convert the image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Create masks for each possible Pac-Man color
    pacman_masks = []
    for pacman_color in pacman_colors:
        lower_pacman = np.array([c - color_tolerance for c in pacman_color], dtype=np.uint8)
        upper_pacman = np.array([c + color_tolerance for c in pacman_color], dtype=np.uint8)
        pacman_mask = cv2.inRange(image_rgb, lower_pacman, upper_pacman)
        pacman_masks.append(pacman_mask)

    # Combine masks for all Pac-Man colors
    combined_pacman_mask = np.bitwise_or.reduce(pacman_masks)

    # Define the background color range
    lower_background = np.array([c - color_tolerance for c in background_color], dtype=np.uint8)
    upper_background = np.array([c + color_tolerance for c in background_color], dtype=np.uint8)

    # Create a mask to identify pixels within the background color range
    background_mask = cv2.inRange(image_rgb, lower_background, upper_background)

    # Exclude the background color from the combined Pac-Man mask
    pacman_mask = cv2.bitwise_and(combined_pacman_mask, cv2.bitwise_not(background_mask))

    # Find contours in the combined Pac-Man mask
    contours, _ = cv2.findContours(pacman_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on area, aspect ratio, and max contour area
    for contour in contours:
        area = cv2.contourArea(contour)
        if min_contour_area < area < max_contour_area:
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h if h != 0 else 0

            if aspect_ratio < max_aspect_ratio:
                # Calculate the centroid (center) of the Pac-Man object
                centroid_x = x + w // 2
                centroid_y = y + h // 2

                # Add the frame and localization to the dataframe
                output_df = pd.concat([output_df, pd.DataFrame([{'Frame': os.path.basename(image_path), 'pacman_X': centroid_x, 'pacman_Y': centroid_y}])], ignore_index=True)

                # Draw a marker at the estimated position
                cv2.circle(image, (centroid_x, centroid_y), 5, (255, 0, 0), -1)

    # Display the result
    # cv2.imshow('Pac-Man Localization', image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

def process_frames(folder_path, filename):
    # Get the total number of files for the progress bar
    total_files = len([filename for filename in os.listdir(folder_path) if filename.endswith('.png')])

    # Create a tqdm progress bar
    for filename in tqdm(sorted(os.listdir(folder_path)), total=total_files):
        if filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            localize_pacman(image_path)
                

    # Save the dataframe to a CSV file
    filename = f"pacman_localization_{i[10:]}.csv"
    output_df.to_csv(filename, index=False)

if __name__ == "__main__":
    # Replace 'your_folder_path' with the actual path to your folder containing PNG frames
    folders = ["ms_pacman/52_RZ_2394668_Aug-10-14-52-42", "ms_pacman/61_RZ_2737165_Aug-14-14-09-12", "ms_pacman/64_RZ_2809807_Aug-15-10-11-14", "ms_pacman/71_RZ_2901714_Aug-16-11-54-21",
               "ms_pacman/76_RZ_3010789_Aug-17-18-01-02",  "ms_pacman/89_RZ_3438636_Aug-22-16-59-30", "ms_pacman/91_RZ_3502739_Aug-23-10-43-13", "ms_pacman/105_RZ_3614646_Aug-24-17-47-26", 
               "ms_pacman/125_JAW_2679477_Dec-07-14-48-06", "ms_pacman/126_JAW_2682560_Dec-07-15-37-29", "ms_pacman/131_KM_3030302_Dec-11-16-12-47", "ms_pacman/137_KM_3115947_Dec-12-15-59-59",
               "ms_pacman/199_RZ_3968652_Dec-04-18-11-43", "ms_pacman/202_RZ_4051421_Dec-05-17-16-28", "ms_pacman/209_RZ_6964528_Jan-08-10-23-46", "ms_pacman/271_RZ_3101375_Mar-20-12-03-54", "ms_pacman/271_RZ_3101375_Mar-20-12-03-54",
               "ms_pacman/273_RZ_3279899_Mar-22-13-39-02", "ms_pacman/378_RZ_2184332_Jul-03-15-07-35", "ms_pacman/467_RZ_3314679_Jul-16-17-05-24", "ms_pacman/496_RZ_3560871_Jul-19-13-28-35"]
    for i in folders: 
        # Define the global dataframe
        output_df = pd.DataFrame(columns=['Frame', 'pacman_X', 'pacman_Y'])
        process_frames(i, i)


100%|██████████| 17793/17793 [00:14<00:00, 1253.34it/s]
